# Building ML-models with feature selection by filter combinations

Preprocessing and feature selection is performed after splitting the data (therefore, only the samples in the training data are available for these steps). 

In [1]:
#import libraries
import pandas as pd
import numpy as np
from IPython.display import display

from functools import partial #to pass parameters to function inside another function
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
from sklearn import svm

In [2]:
#import self-made functions
%run /Users/mariekececilia/Documents/master_thesis_code/methods.ipynb

In [3]:
#set a random seed to make reproducible results (used in mutual information)
seed = 0

## Load and clean data

In [4]:
#load transcriptional and fluxomic data
gexp, flux = load_gerosa()

#clean the transcriptional data (remove duplicates)
gexp, groups = clean_gexp_g(gexp)

#define features
X = gexp

#define targets and extract their fluxes
target_labels = ['R_PPC', 'R_AKGDH', 'R_ICDHyr']
targets = dict()
for target_label in target_labels:
    targets[target_label] = flux[target_label]

## Create models/pipelines

Choose selection parameters:

In [5]:
cov_filters = [0.1, 0.2]
k_numbers = [10,20]

Choose learning algorithms:

In [6]:
algorithms = dict({
    'lr': linear_model.LinearRegression(),
    'svr': svm.SVR(), 
    })

Make combinations:

In [7]:
#make combinations of learning algorithms and filters 
def get_pipelines(algorithm, cov_filters, k_numbers):
    models = dict()
    
    #'baseline' – no selection
    steps = [ 
        ('standardize', StandardScaler()),
        ('m', algorithm)
    ]
    models['noSelection'] = Pipeline(steps=steps)
    
    for cov_filter in cov_filters:
        
        # filter by CoV only
        steps = [
            ('filter', CoVSelector(p = cov_filter)),
            ('standardize', StandardScaler()),
            ('m', algorithm)
        ]
        models['v%.0f' % (cov_filter*100)] = Pipeline(steps=steps)        

        # filter by CoV and SelectKBest
        for k in k_numbers:
            
            #selection by mi
            steps = [
                ('filter', CoVSelector(p = cov_filter)),
                ('selection', SelectKBest(score_func = partial(
                    mutual_info_regression, 
                    random_state=seed), 
                                          k = k)), 
                ('standardize', StandardScaler()),
                ('m', algorithm)
            ]
            models['v%.0f_mi%.0f' % (cov_filter*100, k)] = Pipeline(steps=steps)

            #selection by f
            steps = [
                ('filter', CoVSelector(p = cov_filter)),
                ('selection', SelectKBest(score_func = f_regression, 
                                          k = k)),
                ('standardize', StandardScaler()),
                ('m', algorithm)
            ]
            models['v%.0f_f%.0f' % (cov_filter*100, k)] = Pipeline(steps=steps)
            
    return models

In [8]:
models = dict()

for name, algorithm in algorithms.items():
     models[name]= get_pipelines(algorithm, cov_filters = cov_filters, k_numbers = k_numbers)

## Learning

In [9]:
# evaluate the models and store results in nested dicts: target -> learning algorithm -> scores/preds of all selection combinations
scores = dict()
preds = dict()
for y_name, y in targets.items():
    model_scores = dict()
    model_preds = dict()
    for model_name, pipelines in models.items():
        #get scores
        model_scores[model_name] = compare_models(pipelines, X, y)
        #get predictions
        model_preds[model_name] = get_models_preds(pipelines, X, y)
    scores[y_name] = model_scores
    preds[y_name] = model_preds

### Save data to use in other notebooks

Scores and predictions are visualized in the Notebook 'compare_models', which also merges the results with the results from all other models built.

In [10]:
scores_filter_first = scores
preds_filter_first = preds
%store scores_filter_first
%store preds_filter_first

Stored 'scores_filter_first' (dict)
Stored 'preds_filter_first' (dict)


### Scores and predictions

In [11]:
for y in targets.keys():
    for algorithm in algorithms.keys():
        print()
        print(y + '\t(target)') 
        print(algorithm, '\t(learning algorithm)') 
        print('–> summary of all cv split scores for each combination of selection methods:')
        df = scores[y][algorithm][1].sort_values(by = 'average')
        df.index.name = 'Selection combo'
        df.columns.name = 'Score'
        display(df)
        
        print()
        print(y + '\t(target)') 
        print(algorithm + '\t(learning algorithm)') 
        print('–> all individual cv-split scores for each combination of selection methods:')
        df = scores[y][algorithm][0]
        df.columns.name = 'Selection combo'
        display(df)
        
        print()
        print(y + '\t(target)') 
        print(algorithm + '\t(learning algorithm)') 
        print('–> all predictions for each combination of selection methods (+ actual values):')
        df = preds[y][algorithm]
        df.index.name = 'Test set'
        df.columns.name = 'Selection combo'
        display(df)


R_PPC	(target)
lr 	(learning algorithm)
–> summary of all cv split scores for each combination of selection methods:


Score,average,std
Selection combo,,
v10_f10,0.165893,0.061271
noSelection,0.288090,0.356287
v10,0.291417,0.365301
v20,0.298487,0.365333
v10_f20,0.348834,0.578981
v20_mi20,0.434176,0.558834
v10_mi20,0.439550,0.517082
v20_f20,0.445513,0.771083
v20_f10,0.457395,0.968870



R_PPC	(target)
lr	(learning algorithm)
–> all individual cv-split scores for each combination of selection methods:


Selection combo,noSelection,v10,v10_mi10,v10_f10,v10_mi20,v10_f20,v20,v20_mi10,v20_f10,v20_mi20,v20_f20
Test set,,,,,,,,,,,
Acetate,0.220192,0.213017,0.344201,0.158488,0.093540,0.028665,0.257289,0.310719,0.102096,0.006371,0.125843
Fructose,0.298812,0.302572,0.304443,0.185369,0.157177,0.238273,0.328021,0.288418,0.179079,0.159263,0.214565
Galactose,1.188443,1.218405,1.457043,0.115320,1.634755,1.856730,1.222058,3.351341,3.016245,1.757287,2.470376
Glucose,0.046707,0.049225,0.193992,0.128289,0.129831,0.158740,0.059936,0.043552,0.026079,0.089217,0.153046
Glycerol,0.203123,0.203523,0.253199,0.209116,0.796665,0.028455,0.211859,0.253199,0.024107,0.840416,0.077286
Gluconate,0.021594,0.025634,0.051264,0.191467,0.006901,0.069960,0.028152,0.109483,0.062170,0.065311,0.030893
Pyruvate,0.290049,0.282247,1.506541,0.062126,0.553569,0.337785,0.251303,0.437169,0.183477,0.378090,0.365106
Succinate,0.035796,0.036715,0.233522,0.276967,0.143964,0.072062,0.029278,0.208361,0.065907,0.177450,0.126989



R_PPC	(target)
lr	(learning algorithm)
–> all predictions for each combination of selection methods (+ actual values):


...

Score,average,std
Selection combo,,
v20_f20,0.866323,1.417818
v20_f10,0.867634,1.465182
v20_mi10,0.959495,1.650628
v10_f20,0.983437,1.661281
v20_mi20,0.999043,1.705811
v10_f10,1.022578,1.833563
v10_mi20,1.099838,1.842766
v10_mi10,1.154877,1.983930
v10,1.353965,2.080971



R_ICDHyr	(target)
svr	(learning algorithm)
–> all individual cv-split scores for each combination of selection methods:


Selection combo,noSelection,v10,v10_mi10,v10_f10,v10_mi20,v10_f20,v20,v20_mi10,v20_f10,v20_mi20,v20_f20
Test set,,,,,,,,,,,
Acetate,0.366492,0.365448,0.358756,0.323836,0.363548,0.230283,0.358721,0.310678,0.214229,0.331510,0.181262
Fructose,0.330512,0.328893,0.381401,0.268060,0.350166,0.218250,0.318914,0.380533,0.251393,0.376369,0.214738
Galactose,6.592371,6.602266,6.255335,5.807384,5.818205,5.228016,6.646155,5.218304,4.645305,5.413463,4.484874
Glucose,0.122997,0.115938,0.014275,0.174520,0.017098,0.082425,0.098029,0.014275,0.174520,0.024904,0.054093
Glycerol,0.491079,0.492230,0.011053,0.028162,0.021608,0.149713,0.496921,0.011053,0.028162,0.028571,0.147182
Gluconate,2.193906,2.196801,1.567011,1.055972,1.533377,1.464949,2.200395,1.212730,1.125751,1.182305,1.284413
Pyruvate,0.620669,0.619482,0.533547,0.455779,0.615521,0.454278,0.614531,0.462898,0.455779,0.546581,0.454278
Succinate,0.114616,0.110663,0.117641,0.066917,0.079178,0.039586,0.104739,0.065491,0.045930,0.088637,0.109744



R_ICDHyr	(target)
svr	(learning algorithm)
–> all predictions for each combination of selection methods (+ actual values):


Selection combo,actual,noSelection,v10,v10_mi10,v10_f10,v10_mi20,v10_f20,v20,v20_mi10,v20_f10,v20_mi20,v20_f20
Test set,,,,,,,,,,,,
Acetate,4.696417,2.975220,2.980123,3.011550,3.175550,2.989046,3.614914,3.011712,3.237342,3.690310,3.139507,3.845135
Fructose,4.566119,3.056961,3.064356,2.824597,3.342126,2.967220,3.569565,3.109921,2.828560,3.418228,2.847572,3.585598
Galactose,0.496177,3.767163,3.772072,3.599933,3.377670,3.383039,3.090201,3.793849,3.085382,2.801073,3.182216,2.721471
Glucose,2.977971,3.344254,3.323230,3.020481,2.458257,3.028890,2.732513,3.269900,3.020481,2.458257,3.052136,2.816884
Glycerol,2.464836,3.675265,3.678104,2.492081,2.534251,2.518096,2.833855,3.689664,2.492081,2.534251,2.535258,2.827616
Gluconate,1.154034,3.685875,3.689216,2.962417,2.372661,2.923603,2.844634,3.693364,2.553565,2.453188,2.518453,2.636290
Pyruvate,7.979320,3.026803,3.036272,3.721982,4.342517,3.067883,4.354490,3.075779,4.285705,4.342517,3.617979,4.354490
Succinate,3.038949,3.387263,3.375250,3.396453,3.242305,3.279568,3.159250,3.357247,2.839927,2.899370,2.769586,2.705442
